In [64]:
from zipfile import ZipFile
from src.utils.helpers import convert_to_category
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

In [7]:
df_client = pd.read_csv('../data/processed/df_client_train_processed.csv')

In [4]:
zip_path = "../data/processed/df_transactions_train_processed.zip"

with ZipFile(zip_path, 'r') as zip_ref:
    csv_filename = zip_ref.namelist()[0]
    with zip_ref.open(csv_filename) as csv_file:
        df_transactions = pd.read_csv(csv_file)

In [8]:
df_client_1 = df_client.copy()
df_transactions_1 = df_transactions.copy()

In [13]:
df_client_1 = convert_to_category(df_client_1, ['district', 'client_catg', 'region'])
df_transactions_1 = convert_to_category(df_transactions_1, ['tarif_type','counter_statue', 'counter_coefficient'])

In [29]:
df_client_1['creation_date'] = pd.to_datetime(df_client_1['creation_date'], format= 'mixed', dayfirst=True)
df_transactions_1['invoice_date'] = pd.to_datetime(df_transactions_1['invoice_date'])

In [39]:
df_transactions_1['lag_1'] = df_transactions_1.groupby('client_id')['consommation_total'].shift(1)

In [40]:
df_transactions_1['variation_conso'] = ((df_transactions_1['consommation_total'] - df_transactions_1['lag_1'])/df_transactions_1['lag_1'])*100

In [42]:
Q1 = df_transactions_1['consommation_total'].quantile(0.25)
Q3 = df_transactions_1['consommation_total'].quantile(0.75)
IQR = Q3 - Q1
borne_inf = Q1 - 1.5 * IQR
borne_sup = Q3 + 1.5 * IQR

df_transactions_1['outlier_conso_total'] = ((df_transactions_1['consommation_total'] < borne_inf) | (df_transactions_1['consommation_total'] > borne_sup)).astype(int)

Q1_var = df_transactions_1['variation_conso'].quantile(0.25)
Q3_var = df_transactions_1['variation_conso'].quantile(0.75)
IQR_var = Q3_var - Q1_var
borne_inf_var = Q1_var - 1.5 * IQR_var
borne_sup_var = Q3_var + 1.5 * IQR_var

df_transactions_1['outlier_variation'] = ((df_transactions_1['variation_conso'] < borne_inf_var) | (df_transactions_1['variation_conso'] > borne_sup_var)).astype(int)

In [48]:
df_transactions_1['is_null_conso'] = (df_transactions_1['consommation_total'] == 0).astype(int)

In [50]:
df_transactions_1['part_conso_level_1'] = (df_transactions_1['consommation_level_1'] / df_transactions_1['consommation_total'])*100
df_transactions_1['part_conso_level_2'] = (df_transactions_1['consommation_level_2'] / df_transactions_1['consommation_total'])*100
df_transactions_1['part_conso_level_3'] = (df_transactions_1['consommation_level_3'] / df_transactions_1['consommation_total'])*100
df_transactions_1['part_conso_level_4'] = (df_transactions_1['consommation_level_4'] / df_transactions_1['consommation_total'])*100

In [52]:
df_transactions_1['dominant_level'] = df_transactions_1[['part_conso_level_1', 'part_conso_level_2', 'part_conso_level_3', 'part_conso_level_4']].idxmax(axis=1)

C:\Users\djidj\AppData\Local\Temp\ipykernel_9156\1237248556.py:1: FutureWarning: The behavior of DataFrame.idxmax with all-NA values, or any-NA and skipna=False, is deprecated. In a future version this will raise ValueError
  df_transactions_1['dominant_level'] = df_transactions_1[['part_conso_level_1', 'part_conso_level_2', 'part_conso_level_3', 'part_conso_level_4']].idxmax(axis=1)


In [67]:
df_transactions_1 = convert_to_category(df_transactions_1, ['outlier_conso_total', 'outlier_variation', 'is_null_conso','dominant_level'])

In [ ]:
df_transactions_1['dominant_level'] = df_transactions_1['dominant_level'].cat.set_categories(['part_conso_level_1', 'part_conso_level_2', 'part_conso_level_3', 'part_conso_level_4'])
df_transactions_1['dominant_level'] = df_transactions_1['dominant_level'].cat.rename_categories({'part_conso_level_1':1, 'part_conso_level_2':2, 'part_conso_level_3':3, 'part_conso_level_4':4})

In [72]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_columns = ['tarif_type', 'counter_statue', 'counter_coefficient', 'outlier_conso_total', 'outlier_variation', 'is_null_conso', 'dominant_level']
encoded = encoder.fit_transform(df_transactions_1[cat_columns])


In [75]:
encoded_df = pd.DataFrame(encoded, columns=encoder.get_feature_names_out(cat_columns))

df_transactions_final = pd.concat([df_transactions_1.drop(cat_columns, axis=1), encoded_df], axis=1)

In [77]:
df_transactions_final.columns

Index(['client_id', 'invoice_date', 'counter_number', 'counter_code',
       'reading_remarque', 'consommation_level_1', 'consommation_level_2',
       'consommation_level_3', 'consommation_level_4', 'old_index',
       'new_index', 'months_number', 'consommation_total', 'conso_level',
       'ecart', 'conso_incoherente', 'lag_1', 'variation_conso',
       'part_conso_level_1', 'part_conso_level_2', 'part_conso_level_3',
       'part_conso_level_4', 'tarif_type_9', 'tarif_type_10', 'tarif_type_11',
       'tarif_type_12', 'tarif_type_13', 'tarif_type_14', 'tarif_type_15',
       'tarif_type_18', 'tarif_type_21', 'tarif_type_24', 'tarif_type_29',
       'counter_statue_0', 'counter_statue_1', 'counter_statue_2',
       'counter_statue_3', 'counter_statue_4', 'counter_statue_5',
       'counter_coefficient_1', 'counter_coefficient_20',
       'outlier_conso_total_0', 'outlier_conso_total_1', 'outlier_variation_0',
       'outlier_variation_1', 'is_null_conso_0', 'is_null_conso_1',
      

In [ ]:
df_agg = df_transactions_1.groupby('client_id').agg(
    counter_number = pd.NamedAgg(column='counter_number', aggfunc='max'),
    counter_code = pd.NamedAgg(column='counter_code', aggfunc='max'),
    reading_remarque_min = pd.NamedAgg(column='reading_remarque', aggfunc='min'),
    reading_remarque_max = pd.NamedAgg(column='reading_remarque', aggfunc='max'),
    reading_remarque_std = pd.NamedAgg(column='reading_remarque', aggfunc='std'),
    reading_remarque_mean = pd.NamedAgg(column='reading_remarque', aggfunc='mean'),
    reading_remarque_median = pd.NamedAgg(column='reading_remarque', aggfunc='median'),
    conso_level1_sum = pd.NamedAgg(column='consommation_level_1', aggfunc='sum'),
    conso_level2_sum = pd.NamedAgg(column='consommation_level_2', aggfunc='sum'),
    conso_level3_sum = pd.NamedAgg(column='consommation_level_3', aggfunc='sum'),
    conso_level4_sum = pd.NamedAgg(column='consommation_level_4', aggfunc='sum'),
    conso_min = pd.NamedAgg(column='consommation_total', aggfunc='min'),
    conso_max = pd.NamedAgg(column='consommation_total', aggfunc='max'),
    conso_std = pd.NamedAgg(column='consommation_total', aggfunc='std'),
    conso_mean = pd.NamedAgg(column='consommation_total', aggfunc='mean'),
    conso_median = pd.NamedAgg(column='consommation_total', aggfunc='median')
    ).reset_index()

In [ ]:
df_agg1 = df_transactions_test_v1.groupby('client_id').agg(
    counter_number = pd.NamedAgg(column='counter_number', aggfunc='max'),
    counter_code = pd.NamedAgg(column='counter_code', aggfunc='max'),
    reading_remarque_min = pd.NamedAgg(column='reading_remarque', aggfunc='min'),
    reading_remarque_max = pd.NamedAgg(column='reading_remarque', aggfunc='max'),
    reading_remarque_std = pd.NamedAgg(column='reading_remarque', aggfunc='std'),
    reading_remarque_mean = pd.NamedAgg(column='reading_remarque', aggfunc='mean'),
    reading_remarque_median = pd.NamedAgg(column='reading_remarque', aggfunc='median'),
    conso_level1_sum = pd.NamedAgg(column='consommation_level_1', aggfunc='sum'),
    conso_level2_sum = pd.NamedAgg(column='consommation_level_2', aggfunc='sum'),
    conso_level3_sum = pd.NamedAgg(column='consommation_level_3', aggfunc='sum'),
    conso_level4_sum = pd.NamedAgg(column='consommation_level_4', aggfunc='sum'),
    conso_min = pd.NamedAgg(column='consommation_total', aggfunc='min'),
    conso_max = pd.NamedAgg(column='consommation_total', aggfunc='max'),
    conso_std = pd.NamedAgg(column='consommation_total', aggfunc='std'),
    conso_mean = pd.NamedAgg(column='consommation_total', aggfunc='mean'),
    conso_median = pd.NamedAgg(column='consommation_total', aggfunc='median')
    ).reset_index()

In [ ]:
def mode_group(x):
    return x.mode().iloc[0] if not x.mode().empty else None

df_mode = df_transactions_train_v1.groupby('client_id').agg(
    tarif_type = pd.NamedAgg(column='tarif_type', aggfunc=mode_group),
    counter_statue = pd.NamedAgg(column='counter_statue', aggfunc=mode_group),
    counter_coefficient = pd.NamedAgg(column='counter_coefficient', aggfunc=mode_group)
    ).reset_index()

In [ ]:
df_mode1 = df_transactions_test_v1.groupby('client_id').agg(
    tarif_type = pd.NamedAgg(column='tarif_type', aggfunc=mode_group),
    counter_statue = pd.NamedAgg(column='counter_statue', aggfunc=mode_group),
    counter_coefficient = pd.NamedAgg(column='counter_coefficient', aggfunc=mode_group)
    ).reset_index()

In [ ]:

def palier_max_et_compte(df_client):
    paliers = ['consommation_level_1', 'consommation_level_2', 'consommation_level_3', 'consommation_level_4']

    # Création d'un DataFrame booléen pour les franchissements
    franchissements = (df_client[paliers] > 0).sum()
    
    # S'il n'y a aucun franchissement
    if (franchissements == 0).all():
        return pd.Series([None, 0], index=['palier_max', 'nb_max_franchi'])
    
    # Trouver le palier max franchi (ex : palier_3)
    palier_max = franchissements[franchissements > 0].index.map(lambda p: int(p.split('_')[2])).max()
    nom_palier_max = f'consommation_level_{palier_max}'
    
    # Nombre de fois que ce palier max a été franchi
    nb_max = (df_client[nom_palier_max] > 0).sum()
    
    return pd.Series([nom_palier_max, nb_max], index=['palier_max', 'nb_max_franchi'])


In [ ]:
resultat = df_transactions_train_v1.groupby('client_id').apply(palier_max_et_compte).reset_index()

C:\Users\djidj\AppData\Local\Temp\ipykernel_34348\3620131387.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultat = df_transactions_train_v1.groupby('client_id').apply(palier_max_et_compte).reset_index()


In [ ]:
resultat1 = df_transactions_test_v1.groupby('client_id').apply(palier_max_et_compte).reset_index()

C:\Users\djidj\AppData\Local\Temp\ipykernel_34348\4076398852.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  resultat1 = df_transactions_test_v1.groupby('client_id').apply(palier_max_et_compte).reset_index()


In [ ]:
from functools import reduce

dfs = [df_agg, df_mode, resultat]
df_merged = reduce(lambda left, right: pd.merge(left, right, on='client_id', how='left'), dfs)
df_merged.head()

,client_id,counter_number,counter_code,reading_remarque_min,reading_remarque_max,reading_remarque_std,reading_remarque_mean,reading_remarque_median,conso_level1_sum,conso_level2_sum,...,conso_min,conso_max,conso_std,conso_mean,conso_median,tarif_type,counter_statue,counter_coefficient,palier_max,nb_max_franchi
0,train_Client_0,1335667,207,6,9,1.248192,6.971429,6.0,12334,370,...,38,1386,341.553930,362.971429,267.0,11,0,1,consommation_level_2,2.0
1,train_Client_1,678902,203,6,9,1.377097,7.216216,6.0,20629,0,...,190,1207,197.935960,557.540541,520.0,11,0,1,consommation_level_1,37.0
2,train_Client_10,572765,207,6,9,1.258955,7.055556,6.0,14375,682,...,188,3082,646.808386,836.500000,655.5,11,0,1,consommation_level_2,1.0
3,train_Client_100,2078,413,6,9,0.670820,6.150000,6.0,24,0,...,0,15,3.607011,1.200000,0.0,11,0,1,consommation_level_1,4.0
4,train_Client_1000,19575,207,8,9,0.363137,8.857143,9.0,9292,1468,...,124,2382,633.485669,922.642857,770.0,11,0,1,consommation_level_4,2.0


In [ ]:
from functools import reduce

dfs1 = [df_agg1, df_mode1, resultat1]
df_merged1 = reduce(lambda left, right: pd.merge(left, right, on='client_id', how='left'), dfs1)

In [ ]:
df_client_train_final = pd.merge(df_client_train1, df_merged, on='client_id', how='right')

In [ ]:
df_client_test_final = pd.merge(df_client_test, df_merged1, on='client_id', how='right')

In [ ]:
df_client_train_final = deplacer_colonnes(df_client_train_final, 'target',26)

In [ ]:
df_client_test_final = deplacer_colonnes(df_client_test_final, 'client_id',0)